In [36]:
import pexpect
import json
import argparse
import os


class ProofSearch:
    def __init__(self, path_to_repl):

        path_to_repl = os.environ.get('PATH_TO_LEAN_REPL')
        
        self.proc = pexpect.spawn(
            "lake env lean --run REPL/Main.lean", cwd=path_to_repl, encoding="utf-8"
        )
        self.proc.debug = True

    def run_code(self, code, env=None, verbose=False):
        if env:
            command = (
                '{ "cmd" : "' + repr(code)[1:-1] + f'", "env" : {env}' + " }"
            )  # [1:-1] removes single quotes
        else:
            command = (
                '{ "cmd" : "' + repr(code)[1:-1] + '" }'
            )  # [1:-1] removes single quotes

        if verbose:
            print(command)
            
        self.proc.sendline(command)
        self.proc.expect_exact(command + "\r\n")

        self.proc.sendline()
        self.proc.expect_exact("\r\n")
        
        try:
            index = self.proc.expect('env": \d+\}', timeout=20)
            output = self.proc.before + self.proc.match.group()
            if verbose: 
                print(output)
            return json.loads(output)
        except pexpect.exceptions.TIMEOUT:
            print("FAILED DUE TO TIMEOUT")


In [37]:
proofsearch.run_code("import Mathlib.Data.List.Basic\ndef f := 2", verbose=True)

{ "cmd" : "import Mathlib.Data.List.Basic\ndef f := 2" }
{"env": 1}


{'env': 1}

In [38]:
proofsearch.run_code("example : 2 = 2 := by", verbose=True)

{ "cmd" : "example : 2 = 2 := by" }
{"messages":
 [{"severity": "error",
   "pos": {"line": 1, "column": 21},
   "endPos": null,
   "data": "unexpected end of input; expected '{'"},
  {"severity": "error",
   "pos": {"line": 1, "column": 19},
   "endPos": {"line": 1, "column": 21},
   "data": "unsolved goals\n⊢ 2 = 2"}],
 "env": 2}


{'messages': [{'severity': 'error',
   'pos': {'line': 1, 'column': 21},
   'endPos': None,
   'data': "unexpected end of input; expected '{'"},
  {'severity': 'error',
   'pos': {'line': 1, 'column': 19},
   'endPos': {'line': 1, 'column': 21},
   'data': 'unsolved goals\n⊢ 2 = 2'}],
 'env': 2}

In [39]:
proofsearch.run_code("example : 2 = 3 := rfl", verbose=True)

{ "cmd" : "example : 2 = 3 := rfl" }
{"messages":
 [{"severity": "error",
   "pos": {"line": 1, "column": 19},
   "endPos": {"line": 1, "column": 22},
   "data":
   "type mismatch\n  rfl\nhas type\n  2 = 2 : Prop\nbut is expected to have type\n  2 = 3 : Prop"}],
 "env": 3}


{'messages': [{'severity': 'error',
   'pos': {'line': 1, 'column': 19},
   'endPos': {'line': 1, 'column': 22},
   'data': 'type mismatch\n  rfl\nhas type\n  2 = 2 : Prop\nbut is expected to have type\n  2 = 3 : Prop'}],
 'env': 3}

In [40]:
feedback = proofsearch.run_code("def f := 37", verbose=True)

{ "cmd" : "def f := 37" }
{"env": 4}


In [41]:
env = feedback["env"]
proofsearch.run_code("#check (rfl: f = 37)", env=env, verbose=True)

{ "cmd" : "#check (rfl: f = 37)", "env" : 4 }
{"messages":
 [{"severity": "info",
   "pos": {"line": 1, "column": 0},
   "endPos": {"line": 1, "column": 6},
   "data": "rfl : f = f"}],
 "env": 5}


{'messages': [{'severity': 'info',
   'pos': {'line': 1, 'column': 0},
   'endPos': {'line': 1, 'column': 6},
   'data': 'rfl : f = f'}],
 'env': 5}

In [13]:
path = os.environ.get("PATH_TO_LEAN_REPL")

print("lean repl path: ", path)

proofsearch = ProofSearch(path)
state = proofsearch.run_code("example : 2 = 2 := by", verbose=True)

lean repl path:  /localscratch/hsun409/github/repl
{ "cmd" : "example : 2 = 2 := by" }
{"messages":
 [{"severity": "error",
   "pos": {"line": 1, "column": 21},
   "endPos": null,
   "data": "unexpected end of input; expected '{'"},
  {"severity": "error",
   "pos": {"line": 1, "column": 19},
   "endPos": {"line": 1, "column": 21},
   "data": "unsolved goals\n⊢ 2 = 2"}],
 "env": 0}


In [42]:
code = '''
import Mathlib.Data.List.Basic

def f := 2

-- Example theorem or lemma
theorem example_theorem : f > 0 :=
begin
  -- Proof goes here
  sorry -- This is a placeholder
end
'''

In [53]:
code = '''
import Mathlib.Data.List.Basic\ndef f := 2\ntheorem example_theorem : f > 0 := sorry
'''

In [54]:
state = proofsearch.run_code(code, verbose=True)

{ "cmd" : "\nimport Mathlib.Data.List.Basic\ndef f := 2\ntheorem example_theorem : f > 0 := sorry\n" }
{"sorries":
 [{"proofState": 0,
   "pos": {"line": 4, "column": 35},
   "goal": "⊢ f > 0",
   "endPos": {"line": 4, "column": 40}}],
 "messages":
 [{"severity": "warning",
   "pos": {"line": 4, "column": 8},
   "endPos": {"line": 4, "column": 23},
   "data": "declaration uses 'sorry'"}],
 "env": 10}


In [55]:
state

{'sorries': [{'proofState': 0,
   'pos': {'line': 4, 'column': 35},
   'goal': '⊢ f > 0',
   'endPos': {'line': 4, 'column': 40}}],
 'messages': [{'severity': 'warning',
   'pos': {'line': 4, 'column': 8},
   'endPos': {'line': 4, 'column': 23},
   'data': "declaration uses 'sorry'"}],
 'env': 10}

In [27]:
state['messages'][1]['data'].startswith('unsolved goals\n')

True

In [30]:
for msg in state['messages']:
    data = msg['data']
    if data.startswith('unsolved goals\n'):
        print(data)

unsolved goals
⊢ 2 = 2


In [33]:


def get_goal(state):
    goal = None
    for msg in state['messages']:
        if msg['data'].startswith('unsolved goals\n'):
            goal = '\n'.join(msg['data'].split('\n')[1:])
        # elif msg['severity'] == 'error':
        #     return None
    return goal

In [34]:
print(get_goal(state))

⊢ 2 = 2
